| [09_dialogue/01_对话模型.ipynb](https://github.com/shibing624/nlp-tutorial/tree/main/09_dialogue/01_对话模型.ipynb)  | 基于transformers的Bert阅读理解任务  |[Open In Colab](https://colab.research.google.com/github/shibing624/nlp-tutorial/blob/main/09_dialogue/01_对话模型.ipynb) |


# 对话模型


使用transformers的Bert模型完成阅读理解任务。

In [1]:
!pip install transformers

In [7]:
import os

from transformers import pipeline
from transformers import AutoModelForQuestionAnswering, BertTokenizer
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
bert_model = 'bert-base-chinese'
print(bert_model)
# model = AutoModelForQuestionAnswering.from_pretrained(bert_model_dir)
# tokenizer = BertTokenizer.from_pretrained(bert_model_dir)
nlp = pipeline("question-answering",
               model=bert_model,
               tokenizer=bert_model,
               device=-1,  # gpu device id
               )
context = r"""
大家好，我是张亮，目前任职当当架构部架构师一职，也是高可用架构群的一员。我为大家提供了一份imagenet数据集，希望能够为图像分类任务做点贡献。
"""

# context = ' '.join(list(context))

result = nlp(question="张亮在哪里任职?", context=context)
print(
    f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")
result = nlp(question="张亮为图像分类提供了什么数据集?", context=context)
print(
    f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")


bert-base-chinese


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-chinese a

Answer: '图像分类', score: 0.0008, start: 63, end: 67
Answer: '可', score: 0.0007, start: 28, end: 29



# Custom Predict

In [6]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
bert_model = 'bert-base-uncased'
# tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
# model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained(bert_model)
tokenizer = BertTokenizer.from_pretrained(bert_model)

text = r"""
Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""
questions = [
    "How many pretrained models are available in Transformers?",
    "What does Transformers provide?",
    "Transformers provides interoperability between which frameworks?",
]
for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    print(f'inputs:{inputs}')
    input_ids = inputs["input_ids"].tolist()[0]
    outputs = model(**inputs)
    print(outputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits
    answer_start = torch.argmax(
        answer_start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    print(f"Question: {question}")
    print(f"Answer: {answer}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

inputs:{'input_ids': tensor([[  101,  2129,  2116,  3653, 23654,  2098,  4275,  2024,  2800,  1999,
         19081,  1029,   102, 19081,  1006,  3839,  2124,  2004,  1052, 22123,
          2953,  2818,  1011, 19081,  1998,  1052, 22123,  2953,  2818,  1011,
          3653, 23654,  2098,  1011, 14324,  1007,  3640,  2236,  1011,  3800,
          4294,  2015,  1006, 14324,  1010, 14246,  2102,  1011,  1016,  1010,
         23455,  1010, 28712,  2213,  1010,  4487, 16643, 23373,  1010, 28712,
          7159,  1529,  1007,  2005,  3019,  2653,  4824,  1006, 17953,  2226,
          1007,  1998,  3019,  2653,  4245,  1006, 17953,  2290,  1007,  2007,
          2058,  3590,  1009,  3653, 23654,  2098,  4275,  1999,  2531,  1009,
          4155,  1998,  2784,  6970, 25918,  8010,  2090, 23435, 12314,  1016,
          1012,  1014,  1998,  1052, 22123,  2953,  2818,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-1.2878e-01,  3.0782e-01, -2.4559e-01, -1.4951e-01, -1.1949e-01,
         -1.6474e-01, -2.4681e-01,  2.1168e-01,  2.8502e-01, -2.6541e-01,
         -3.9923e-01, -7.1881e-02,  5.4817e-01, -4.5537e-01, -2.4568e-01,
         -5.1151e-01, -4.4427e-01, -1.2528e-01,  7.1618e-02, -3.4776e-01,
          2.7170e-01, -1.9582e-01,  4.6603e-01, -6.5703e-02, -1.9252e-02,
          9.8797e-02, -3.3767e-01,  1.2233e-01, -2.0983e-01,  4.6932e-01,
          6.0442e-02, -7.5030e-02, -3.2702e-01, -2.5363e-01,  9.7604e-03,
         -2.9355e-01, -3.0845e-01, -5.5540e-02, -1.4168e-01, -3.8999e-03,
         -8.2347e-02, -3.4380e-01, -4.9974e-03, -1.0133e-01, -6.8717e-02,
          1.5717e-01, -1.1907e-01, -2.6423e-01, -7.4110e-02,  7.7396e-02,
         -8.3980e-02,  1.2772e-01, -2.6426e-01, -1.6226e-01,  2.6700e-01,
         -1.3828e-01, -5.9531e-02, -9.3357e-02,  2.8573e-01, -2.6597e-01,
         -1.0377e-01,  3.4796e-02, -3.9672e-01,  5.3762e-01